In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
archivo1 = "VictimasMA_202309.xlsx"
archivo2 = "CasosMA_202309.xlsx"
victimas_df= pd.read_excel(archivo1)
casos_df= pd.read_excel(archivo2)


FileNotFoundError: [Errno 2] No such file or directory: 'VictimasMA_202309.xlsx'

In [19]:
def clean_columnms(cadena):
    """Funcion para limpiar las columnas de un dataframe

    Args:
        cadena (str): nombre de columnba

    Returns:
        str: columna nueva
    """
    cadena = cadena.replace(" ", "_").lower()
 
    return cadena

#Determinar las columnas de cada base de datos
columns_victimas = victimas_df.columns
columns_casos = casos_df.columns

#Limpiamos los nombres de las columbas
new_columns_victimas = [clean_columnms(columns_victimas) for columns_victimas in columns_victimas]
new_columns_casos = [clean_columnms(columns_casos) for columns_casos in columns_casos]
new_columns_victimas = [ unidecode(new_columns_victimas) for new_columns_victimas in new_columns_victimas] #quita las tildes, punto, etc
new_columns_casos = [ unidecode(new_columns_casos) for new_columns_casos in new_columns_casos]

victimas_df.columns = new_columns_victimas
casos_df.columns = new_columns_casos

print(victimas_df.columns)
print(casos_df.columns)
#print(pd.unique(casos_df['violencia_basada_en_genero']))
#pd.unique(victimas_df['descripcion_fuerza_o_grupo_armado_organizado_al_que_pertenece_el_combatiente'])


Index(['id_caso', 'divipola', 'municipio', 'departamento', 'ano', 'mes', 'dia',
       'id_persona', 'sexo', 'etnia', 'ocupacion', 'calidad_victima_',
       'tipo_poblacion', 'militancia', 'grupo_armado', 'descrip_grupo_armado',
       'edad', 'latitud', 'longitud'],
      dtype='object')
Index(['id_caso', 'id_caso_relacionado', 'ano', 'mes', 'dia', 'divipola',
       'municipio', 'departamento', 'region', 'modalidad',
       'presunto_responsable', 'presunto_grupo_respon',
       'despojo_forzoso_tierras_', 'amenaza_intimidacion', 'ataque_medicos',
       'restriccion_movilidad', 'desplazamiento_forzado', 'extorsion',
       'lesionados_civiles', 'pillaje', 'tortura', 'vbg', 'hecho_simultaneo',
       'total_victimas', 'tipo_de_armas', 'latitud', 'longitud'],
      dtype='object')


In [20]:
victimas_df.rename(columns={  'codigo_dane_de_municipio':'divipola',
                                     'calidad_de_la_victima_o_la_baja' : 'calidad_victima ',
                                     'tipo_de_poblacion_vulnerable' : 'tipo_poblacion',
                                     'militante_politico' : 'militancia',
                                     'fuerza_o_grupo_armado_organizado_al_que_pertenece_el_combatiente': 'grupo_armado',
                                     'descripcion_fuerza_o_grupo_armado_organizado_al_que_pertenece_el_combatiente':'descrip_grupo_armado',
                                     'situacion_actual_de_la_victima': 'situacion_actual_vic'
                                     
                                     },inplace=True)
victimas_df.columns


casos_df.rename(columns={'codigo_dane_de_municipio':'divipola',
                                     'abandono_o_despojo_forzado_de_tierras' : 'despojo_forzoso_tierras ',
                                     'amenaza_o_intimidacion' : 'amenaza_intimidacion',
                                     'confinamiento_o_restriccion_a_la_movilidad' : 'restriccion_movilidad',
                                     'ataque_contra_mision_medica': 'ataque_medicos',
                                     'descripcion_presunto_responsable':'presunto_grupo_respon',
                                     'violencia_basada_en_genero': 'VBG',
                                     'otro_hecho_simultaneo': 'hecho_simultaneo',
                                     'total_de_victimas_del_caso':'total_victimas'
                                     },inplace=True)

victimas_df['divipola'] = pd.to_numeric(victimas_df['divipola'], errors='coerce')
casos_df['divipola'] = pd.to_numeric(victimas_df['divipola'], errors='coerce')


print(victimas_df.columns)
print(casos_df.columns)

#print(casos_df.tail(10).info())
#print(casos_df.dtypes)


Index(['id_caso', 'divipola', 'municipio', 'departamento', 'ano', 'mes', 'dia',
       'id_persona', 'sexo', 'etnia', 'ocupacion', 'calidad_victima_',
       'tipo_poblacion', 'militancia', 'grupo_armado', 'descrip_grupo_armado',
       'edad', 'latitud', 'longitud'],
      dtype='object')
Index(['id_caso', 'id_caso_relacionado', 'ano', 'mes', 'dia', 'divipola',
       'municipio', 'departamento', 'region', 'modalidad',
       'presunto_responsable', 'presunto_grupo_respon',
       'despojo_forzoso_tierras_', 'amenaza_intimidacion', 'ataque_medicos',
       'restriccion_movilidad', 'desplazamiento_forzado', 'extorsion',
       'lesionados_civiles', 'pillaje', 'tortura', 'vbg', 'hecho_simultaneo',
       'total_victimas', 'tipo_de_armas', 'latitud', 'longitud'],
      dtype='object')


In [ ]:
# Define a function to identify missing or placeholder values
def identify_missing_or_placeholder(df):
    # Placeholder values that might be considered as missing data
    placeholders = ["SIN INFORMACION", "DESCONOCIDO", "ND", "0", 0, "NaN", ""]

    missing_values = df.isnull().mean() * 100

    placeholder_values = df.apply(lambda col: col.isin(placeholders).mean()) * 100

    missing_info = pd.DataFrame({'missing_percentage': missing_values, 'placeholder_percentage': placeholder_values})
    missing_info['total_impacted'] = missing_info.sum(axis=1)

    return missing_info.sort_values(by='total_impacted', ascending=False)

missing_info_casos = identify_missing_or_placeholder(casos_df)
missing_info_victimas = identify_missing_or_placeholder(victimas_df)


# Eliminar columnas con 95% missing/placeholder data de ambos archivos
threshold_percentage = 95
casos_df_cleaned = casos_df.drop(columns=missing_info_casos[missing_info_casos['total_impacted'] > threshold_percentage].index)
victimas_df_cleaned = victimas_df.drop(columns=missing_info_victimas[missing_info_victimas['total_impacted'] > threshold_percentage].index)

#Organizar datos que son marcados por defecto
for column in ['ano', 'mes', 'dia']:
    casos_df_cleaned[column] = np.where(casos_df_cleaned[column] == 0, pd.NA, casos_df_cleaned[column])
    victimas_df_cleaned[column] = np.where(victimas_df_cleaned[column] == 0, pd.NA, victimas_df_cleaned[column])

updated_info = {
    "Casos Dataset Shape": casos_df_cleaned.shape,
    "Victimas Dataset Shape": victimas_df_cleaned.shape,
    "Sample Año (Casos)": casos_df_cleaned['ano'].dropna().unique()[:5],
    "Sample Año (Victimas)": victimas_df_cleaned['ano'].dropna().unique()[:5]
}

casos_df_nodup = casos_df_cleaned.drop_duplicates(casos_df_cleaned.columns.difference(['id_caso']))
victimas_df_nodup = victimas_df_cleaned.drop_duplicates(victimas_df_cleaned.columns.difference(['id_persona']))


unique_modalidades = casos_df_nodup['modalidad'].unique()
unique_deptos = casos_df_nodup['departamento'].unique()

validation_info = {
    "Casos Dataset Shape Before": casos_df_cleaned.shape,
    "Casos Dataset Shape After Duplicate Removal": casos_df_nodup.shape,
    "Victimas Dataset Shape Before": victimas_df_cleaned.shape,
    "Victimas Dataset Shape After Duplicate Removal": victimas_df_nodup.shape,
    "Unique Modalidades": unique_modalidades[:5],  # Display a sample for brevity
    "Unique Deptos": unique_deptos[:5]
}

print(casos_df['ano'], missing_info_casos)


In [16]:
print(casos_df['VBG'])

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4319    NaN
4320    NaN
4321    NaN
4322    NaN
4323    NaN
Name: VBG, Length: 4324, dtype: object


ANALSIS INICIAL EXPLORATORIO DE LOS DATOS: respondemos a las pregunta que tengamos

In [ ]:
# casos por año
cases_yearly = casos_df['ano'].value_counts().sort_index()
#victimas por año
victims_yearly = victimas_df['ano'].value_counts().sort_index()

# dataframe a visualizar
yearly_data = pd.DataFrame({
    'Cases': cases_yearly,
    'Victims': victims_yearly
}).fillna(0)
yearly_data=yearly_data.drop(0) #me tocó eliminar la fila cero
print(yearly_data)
# Visualization of yearly data
plt.figure(figsize=(14, 7))
plt.plot(yearly_data.index, yearly_data['Cases'], label='Cases', marker='o')
plt.plot(yearly_data.index, yearly_data['Victims'], label='Victims', marker='x')
plt.title('Yearly Counts of Cases and Victims')
plt.xlabel('Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Hay algun mes con mayor incidencia en masacres? 

casos_por_mes = casos_df.groupby('mes').size()

print("Cantidad de casos registrados por mes:")
print(casos_por_mes)

plt.figure(figsize=(10, 6))
casos_por_mes.plot(kind='bar', color='skyblue')
plt.title('Cantidad de casos registrados por mes')
plt.xlabel('Mes')
plt.ylabel('Cantidad de casos')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Cuál es la mayo situación de VBG? 
casos_por_vbg = casos_df.groupby('VBG').size()

print("Cantidad de casos VBG:")
print(casos_por_vbg)

plt.figure(figsize=(10, 6))
casos_por_mes.plot(kind='bar', color='skyblue')
plt.title('Cantidad de casos VBG')
plt.xlabel('Tipo')
plt.ylabel('Cantidad de casos')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Cuál caldiad de la victima a la baja? 
victimas_calidad = victimas_df.groupby('calidad_victima_').size()

print("victimas_calidad:")
print(victimas_calidad)

plt.figure(figsize=(10, 6))
plt.pie(victimas_calidad, labels=victimas_calidad.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.tab20.colors)
plt.title('victimas_calidad')
plt.xlabel('Tipo')
plt.ylabel('Cantidad de casos')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Cuál caldiad de la victima a la baja? 
victimas_calidad = victimas_df.groupby('calidad_victima_').size()

print("victimas_calidad:")
print(victimas_calidad)

plt.figure(figsize=(10, 6))
plt.pie(victimas_calidad, labels=victimas_calidad.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.tab20.colors)
plt.title('victimas_calidad')
plt.xlabel('Tipo')
plt.ylabel('Cantidad de casos')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()